In [1]:
import sys
import os

# Set PYTHONPATH dynamically
python_path = r'C:\Users\AFreitas\Documents\git_projects\flower\baselines\niid_bench'
sys.path.append(python_path)

# Verify it has been added
print("Current PYTHONPATH:", sys.path)

Current PYTHONPATH: ['C:\\Users\\AFreitas\\AppData\\Local\\anaconda3\\envs\\flwr-niid-bench\\python310.zip', 'C:\\Users\\AFreitas\\AppData\\Local\\anaconda3\\envs\\flwr-niid-bench\\DLLs', 'C:\\Users\\AFreitas\\AppData\\Local\\anaconda3\\envs\\flwr-niid-bench\\lib', 'C:\\Users\\AFreitas\\AppData\\Local\\anaconda3\\envs\\flwr-niid-bench', 'c:\\Users\\AFreitas\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\niid-bench-rTMtSNmn-py3.10', '', 'c:\\Users\\AFreitas\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\niid-bench-rTMtSNmn-py3.10\\lib\\site-packages', 'C:\\Users\\AFreitas\\Documents\\git_projects\\flower\\baselines\\niid_bench', 'c:\\Users\\AFreitas\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\niid-bench-rTMtSNmn-py3.10\\lib\\site-packages\\win32', 'c:\\Users\\AFreitas\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\niid-bench-rTMtSNmn-py3.10\\lib\\site-packages\\win32\\lib', 'c:\\Users\\AFreitas\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\niid-bench-rTMtSNmn-py3.10\\lib\\site-packa

In [14]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
from niid_bench.models import CNN, train_fedavg
from torch.utils.data import DataLoader, TensorDataset
%matplotlib inline

In [5]:
os.chdir(r'C:\Users\AFreitas\Documents\git_projects\flower\baselines\niid_bench')
print("New Working Directory:", os.getcwd())

New Working Directory: C:\Users\AFreitas\Documents\git_projects\flower\baselines\niid_bench


In [25]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations (convert to tensor and normalize)
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts the image to PyTorch tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

# Download the training and testing datasets
train_dataset = datasets.SVHN(root='./data/SVHN', split='train', transform=transform, download=True)
test_dataset = datasets.SVHN(root='./data/SVHN', split='test', transform=transform, download=True)

# Create data loaders for batching
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Verify dataset loading
print(f"Train Dataset Size: {len(train_dataset)}")
print(f"Test Dataset Size: {len(test_dataset)}")

# Access a sample from the dataset
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Print the shape of an image tensor and its label
print(f"Image shape: {images[0].shape}, Label: {labels[0].item()}")


Using downloaded and verified file: ./data/SVHN\train_32x32.mat
Using downloaded and verified file: ./data/SVHN\test_32x32.mat
Train Dataset Size: 73257
Test Dataset Size: 26032
Image shape: torch.Size([3, 32, 32]), Label: 1


In [22]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the CNN with appropriate parameters
model = CNN(input_dim=400, hidden_dims=[120, 84], num_classes=10).to(device)

In [26]:
# Train the CNN on the SVHN dataset
train_fedavg(
    net=model,
    trainloader=train_loader,
    device=device,
    epochs=3,         # Number of epochs
    learning_rate=0.01,  # Learning rate
    momentum=0.9,       # Momentum for SGD optimizer
    weight_decay=1e-4   # Weight decay for regularization
)

References

https://www.kaggle.com/code/dimitriosroussis/svhn-classification-with-cnn-keras-96-acc